In [1]:
from gdal import ogr
import ogr2ogr
import os
import gdal

# Read the data

In [2]:
natura_regions = ogr.Open('Input_data_Milos/Natura_Milos/Natura_Milos.shp', 1)
natura_regions_layer = natura_regions.GetLayer()

In [3]:
# Natura regions of the island include include some regions 

In [4]:
towns = ogr.Open('Input_data_Milos/Oikismoi_Milos/Oikismoi_Milos.shp', 1)
towns_layer = towns.GetLayer()

In [5]:
coastline = ogr.Open('Input_data_Milos/coastline_Milos/coastline_Milos.shp', 1)
coastline_layer = coastline.GetLayer()

In [6]:
roads = ogr.Open('Input_data_Milos/Roads/merge_roads_Milos.shp', 1)
roads_layer = coastline.GetLayer()

In [7]:
import geopandas as gpd

In [8]:
coast = gpd.read_file('Input_data_Milos/coastline_Milos/coastline_Milos.shp')
roads = gpd.read_file('Input_data_Milos/Roads/merge_roads_Milos.shp')

# Function for buffer

In [9]:
def createBuffer(inputfn, outputBufferfn, bufferDist):
    inputds = ogr.Open(inputfn)
    inputlyr = inputds.GetLayer()
        
    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
#     if os.path.exists(outputBufferfn):
#         shpdriver.DeleteDataSource(outputBufferfn)
    outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
    bufferlyr = outputBufferds.CreateLayer(outputBufferfn, inputlyr.GetSpatialRef(),
                                           geom_type=ogr.wkbPolygon)
    featureDefn = bufferlyr.GetLayerDefn()

    for feature in inputlyr:
        ingeom = feature.GetGeometryRef()
        geomBuffer = ingeom.Buffer(bufferDist)

        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geomBuffer)
        bufferlyr.CreateFeature(outFeature)
        feature = None
        
    outputBufferds = bufferlyr = None
        
    feature = None

# Buffer Calculation

In [10]:
# Towns buffer
createBuffer('Input_data_Milos/Oikismoi_Milos/Oikismoi_Milos.shp', 'Produced/towns_buffer.shp', 300)
# Coastline buffer
createBuffer('Input_data_Milos/coastline_Milos/coastline_Milos.shp', 'Produced/coastline_buffer.shp', 300)
# Natura 2000 buffer
createBuffer('Input_data_Milos/Natura_Milos/Natura_Milos.shp', 'Produced/natura_buffer.shp', 800)

# Land uses

In [11]:
os.system('''ogr2ogr -sql "SELECT * FROM CLC_Milos \
        WHERE CODE_18!='321' AND CODE_18!='323' AND CODE_18!='333'" \
        Produced/Selected_land_uses.shp  Input_data_Milos/Corine_Land_Cover_Milos/CLC_Milos.shp''')

0

0 means that the script run successfully whereas 1 the other way around.

# Roads selection (secondary and tertiary)

In [12]:
# os.system('''ogr2ogr -sql "SELECT * FROM merge_roads_Milos WHERE highway='secondary' OR highway='tertiary'" \
# Selected_roads.shp  merge_roads_Milos.shp''')


os.system('''ogr2ogr -sql "SELECT * FROM merge_roads_Milos \
        WHERE highway='secondary' OR highway='tertiary'" \
        Produced/Selected_roads.shp  Input_data_Milos/Roads/merge_roads_Milos.shp''')

# Buffer for selected_roads (distance 2km)
createBuffer('Produced/Selected_roads.shp', 'Produced/Selected_roads_buffer.shp', 400)


# Merge excluded areas

In [13]:
def merge():
    ogr2ogr.main(["","-f", "ESRI Shapefile", "Produced/merged.shp", "Produced/coastline_buffer.shp"])
    ogr2ogr.main(["","-f", "ESRI Shapefile", "-append", "-update", 'Produced/merged.shp', "Produced/towns_buffer.shp"])
    ogr2ogr.main(["","-f", "ESRI Shapefile", "-append", "-update", 'Produced/merged.shp', "Produced/natura_buffer.shp"])
    
#     ogr2ogr -f "ESRI Shapefile" merged.shp a.shp+
merge()

# Dissolve merge output to one polygon 

In [14]:
import os

os.system('''ogr2ogr Produced/exclude_dissolved.shp \
Produced/merged.shp -dialect sqlite -sql "SELECT ST_Union(geometry) AS geometry FROM merged ''')

0

# Dissolve land uses

In [15]:
os.system('''ogr2ogr Produced/Selected_land_uses_dis.shp \
Produced/Selected_land_uses.shp -dialect sqlite -sql "SELECT ST_Union(geometry) AS geometry FROM Selected_land_uses''')

0

# Erase gdal

In [16]:
# Erase 
ogr.UseExceptions()

land_uses_dis = ogr.Open('Produced/Selected_land_uses_dis.shp',1)
excluded_areas_dis = ogr.Open('Produced/exclude_dissolved.shp',1)
land_uses_dis_layer = land_uses_dis.GetLayer()
excluded_areas_dis_layer = excluded_areas_dis.GetLayer()

driver = ogr.GetDriverByName('ESRI Shapefile')
erase_outDataSource = driver.CreateDataSource('Produced/erase_output.shp')
srs = land_uses_dis_layer.GetSpatialRef()

# layer name contains some Unicode characters - an excption occured
# how to fix it - https://gis.stackexchange.com/questions/53920/ogr-createlayer-returns-typeerror+

if os.path.exists('Produced/erase_output.shp'):
    driver.DeleteDataSource(erase_outDataSource)
erase_Layer = erase_outDataSource.CreateLayer(str(erase_outDataSource), srs, geom_type = ogr.wkbPolygon)

erase_ds = land_uses_dis_layer.Erase(excluded_areas_dis_layer, erase_Layer)
erase_outDataSource = erase_ds = None

#     bufferlyr = outputBufferds.CreateLayer(outputBufferfn, inputlyr.GetSpatialRef(),
#                                            geom_type=ogr.wkbPolygon)

# Dissolve Selected_roads_buffer

In [17]:
os.system('''ogr2ogr Produced/Selected_roads_buffer_dis.shp \
Produced/Selected_roads_buffer.shp -dialect sqlite -sql "SELECT ST_Union(geometry) AS geometry FROM Selected_roads_buffer''')

0

# Clip

In [18]:
# Input layer
inDataSource = driver.Open('Produced/Selected_roads_buffer_dis.shp', 0)
inLayer = inDataSource.GetLayer()

# clip layer
inClipSource = driver.Open('Produced/erase_output.shp', 0)
inClipLayer = inClipSource.GetLayer()

# output layer
srs = inClipLayer.GetSpatialRef()
outDataSource = driver.CreateDataSource('Produced/clip_layer.shp')
outLayer = outDataSource.CreateLayer('poly', srs, geom_type=ogr.wkbMultiPolygon)

ogr.Layer.Clip(inLayer, inClipLayer, outLayer)

outLayer = outDataSource = None

# Converting multipart to singlepart

In [19]:
def multipoly2poly(in_lyr, out_lyr):
    for in_feat in in_lyr:
        geom = in_feat.GetGeometryRef()
        if geom.GetGeometryName() == 'MULTIPOLYGON':
            for geom_part in geom:
                addPolygon(geom_part.ExportToWkb(), out_lyr)
        else:
            addPolygon(geom.ExportToWkb(), out_lyr)

def addPolygon(simplePolygon, out_lyr):
    featureDefn = out_lyr.GetLayerDefn()
    polygon = ogr.CreateGeometryFromWkb(simplePolygon)
    out_feat = ogr.Feature(featureDefn)
    out_feat.SetGeometry(polygon)
    out_lyr.CreateFeature(out_feat)

clip_file = driver.Open('Produced/clip_layer.shp', 0)
clip_file_layer = clip_file.GetLayer()

outputshp = 'Produced/multiparts.shp'
if os.path.exists(outputshp):
    driver.DeleteDataSource(outputshp)
out_ds = driver.CreateDataSource(outputshp)

srs_2 = clip_file_layer.GetSpatialRef()
out_lyr = out_ds.CreateLayer('Produced/multiparts', srs_2, geom_type=ogr.wkbPolygon)
multipoly2poly(clip_file_layer, out_lyr)

out_ds = out_lyr = None

# Add a column to estimate the area

In [26]:
# read multiparts data

multipart_file = driver.Open('Produced/multiparts.shp', 1)
multipart_layer = multipart_file.GetLayer()

area_field = ogr.FieldDefn('Area', ogr.OFTReal)
area_field.SetWidth(12)
area_field.SetPrecision(3)
multipart_layer.CreateField(area_field)

for feature in multipart_layer:
    geom = feature.GetGeometryRef()
    area_val = geom.GetArea() 
    feature.SetField('Area', area_val)
    multipart_layer.SetFeature(feature)
    
multipart_file = multipart_layer = None

# Select the regions whose area is larger than 1,000,000 m2

In [27]:
os.system('''ogr2ogr -sql "SELECT * FROM multiparts \
        WHERE Area >= 1000000" \
        Produced/multiparts_selected.shp  Produced/multiparts.shp''')

0

There are only two polygons which satisfy these criteria

# The end